In [1]:
import gitpath

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [12]:
file_path = str(gitpath.root()) + '/data/WEC_Perth_49.csv'
df = pd.read_csv(file_path)

print(len(df))

36043


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Assuming df is already loaded
X = df.iloc[:, :98]  # First 98 columns as features
y = df.iloc[:, -1]   # Last column as target

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0)

# Initialize the models
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=0)
gb = GradientBoostingRegressor(n_estimators=100, random_state=0)

# Fit the models on the training data
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)

# Predict on the test data
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_gb = gb.predict(X_test)

# Calculate the MAPE for each model
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb)

# Print MAPE for each model
print(f"Linear Regression MAPE: {mape_lr * 100:.2f}%")
print(f"Random Forest MAPE: {mape_rf * 100:.2f}%")
print(f"Gradient Boosting MAPE: {mape_gb * 100:.2f}%")

# Ensemble the models using Voting Regressor
ensemble_model = VotingRegressor(estimators=[('lr', lr), ('rf', rf), ('gb', gb)])
ensemble_model.fit(X_train, y_train)

# Predict with the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test)

# Calculate the MAPE for the ensemble model
mape_ensemble = mean_absolute_percentage_error(y_test, y_pred_ensemble)

# Print MAPE for the ensemble model
print(f"Ensemble Model MAPE: {mape_ensemble * 100:.2f}%")


Linear Regression MAPE: 0.94%
Random Forest MAPE: 0.28%
Gradient Boosting MAPE: 0.55%
Ensemble Model MAPE: 0.53%


In [19]:
! pip uninstall tensorflow
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Assuming X and y are your features and target variable
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# # Standardize the data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Define the MLP model with TensorFlow/Keras
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1))

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model using multi-core CPUs (TensorFlow automatically uses multiple cores)
# model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# # Evaluate the model on test data
# loss = model.evaluate(X_test_scaled, y_test)
# print("Test Loss:", loss)

^C
